In [1]:
from icecream import ic

## 1. simple

In [44]:
from colorama import Fore

# prefix components:
space = "    "
branch = "│   "
# pointers:
tee = "├── "
last = "└── "


def tree(contents: list, prefix: str = ""):
    # contents each get pointers that are ├── with a final └── :
    pointers = [tee] * (len(contents) - 1) + [last]
    for pointer, d in zip(pointers, contents):
        yield prefix + pointer + Fore.CYAN + d.get("name") + Fore.RESET
        if d.get("c") and isinstance(d.get("c"), list):
            extension = branch if pointer == tee else space
            # i.e. space because last, └── , above so no more |
            yield from tree(d.get("c"), prefix=prefix + extension)


contents = [
    dict(name="p1"),
    dict(
        name="p2",
        c=[dict(name="p21"), dict(name="p22"), dict(name="p23", c=[dict(name="p231")])],
    ),
]
for d in tree(contents):
    print(d)

├── p1
└── p2
    ├── p21
    ├── p22
    └── p23
        └── p231


## 2. more

In [3]:
# https://stackoverflow.com/a/59109706
from itertools import islice
from pathlib import Path
from colorama import Fore

space =  "    "
branch = "│   "
tee =    "├── "
last =   "└── "


def tree(dir_path: Path, level: int = -1, only_dir: bool = False, len_limit: int = 1000):
    """Given a directory Path object print a visual tree structure"""
    dir_path = Path(dir_path)  # accept string coerceable to Path
    files = 0
    directories = 0

    def inner(dir_path: Path, prefix: str = "", level=-1):
        nonlocal files, directories
        if not level:
            return  # 0, stop iterating

        if only_dir:
            contents = [d for d in dir_path.iterdir() if d.is_dir()]
        else:
            contents = list(dir_path.iterdir())

        pointers = [tee] * (len(contents) - 1) + [last]
        for pointer, path in zip(pointers, contents):
            if path.is_dir():
                yield prefix + pointer + Fore.CYAN + path.name + Fore.RESET
                directories += 1
                extension = branch if pointer == tee else space
                yield from inner(path, prefix=prefix + extension, level=level - 1)
            elif not only_dir:
                yield prefix + pointer + path.name
                files += 1

    if not dir_path.is_dir():
        return
    print(dir_path.name)

    iterator = inner(dir_path, level=level)
    for line in islice(iterator, len_limit):
        print(line)

    if next(iterator, None):
        print(f"... len_limit, {len_limit}, reached, counted:")
    print(f"\n{directories} directories" + (f", {files} files" if files else ""))


In [4]:
tree(Path('/tmp/f1'), len_limit=2)

In [15]:
from pathlib import Path


def tree_to_obj(dir_path: Path, level: int = -1, only_dir: bool = False, len_limit: int = 1000):
    """Given a directory Path object build a tree structure"""
    dir_path = Path(dir_path)  # accept string coerceable to Path

    def inner(dir_path: Path, parent: dict):
        for path in list(dir_path.iterdir()):
            _is_dir = path.is_dir()
            content_obj = {
                'name': path.name,
                'directory': _is_dir,
                'fullName': f'{parent.get("fullName")}/{path.name}'
            }
            parent_child = parent.get('children', [])
            parent_child.append(content_obj)
            parent['children'] = parent_child
            if _is_dir:
                inner(path, content_obj)

    if not dir_path.is_dir():
        print(f'{dir_path} is not a valid dir!')
        return
    print(dir_path.name)

    dir_obj = {'name': dir_path.name, 'fullName': dir_path.name, 'directory': True}
    inner(dir_path, dir_obj)
    return dir_obj

In [19]:
from IPython.core import display

In [20]:
display.JSON(tree_to_obj(Path('0-Java')))

0-Java


<IPython.core.display.JSON object>

## 3. for json

In [3]:
import json
resources = json.loads("""
[{"id": 18, "pid": -1, "name": "prj_test", "fullName": "prj_test", "description": "", "children": [{"id": 20, "pid": 18, "name": "aaa.sh", "fullName": "prj_test/aaa.sh", "description": "aaa", "children": [], "type": "FILE", "idValue": "20_0", "dirctory": false, "directory": false}, {"id": 19, "pid": 18, "name": "task1.sh", "fullName": "prj_test/task1.sh", "description": "task1desc", "children": [], "type": "FILE", "idValue": "19_0", "dirctory": false, "directory": false}], "type": "FILE", "idValue": "18_1", "dirctory": true, "directory": true}, {"id": 22, "pid": -1, "name": "prj_yovole_ubp_ds", "fullName": "prj_yovole_ubp_ds", "description": "客户等级分级ubp project", "children": [{"id": 26, "pid": 22, "name": "test-cal-fact-core-asset-detail-hbase.sql", "fullName": "prj_yovole_ubp_ds/test-cal-fact-core-asset-detail-hbase.sql", "description": "", "children": [], "type": "FILE", "idValue": "26_0", "dirctory": false, "directory": false}, {"id": 28, "pid": 22, "name": "test-cal-fact-customer-hbase.sql", "fullName": "prj_yovole_ubp_ds/test-cal-fact-customer-hbase.sql", "description": "", "children": [], "type": "FILE", "idValue": "28_0", "dirctory": false, "directory": false}, {"id": 24, "pid": 22, "name": "test-create-external-table.sql", "fullName": "prj_yovole_ubp_ds/test-create-external-table.sql", "description": "", "children": [], "type": "FILE", "idValue": "24_0", "dirctory": false, "directory": false}, {"id": 27, "pid": 22, "name": "ubp_cal_fact_cdh_dmw_tables.sh", "fullName": "prj_yovole_ubp_ds/ubp_cal_fact_cdh_dmw_tables.sh", "description": "", "children": [], "type": "FILE", "idValue": "27_0", "dirctory": false, "directory": false}, {"id": 25, "pid": 22, "name": "ubp_cal_fact_cdh_ods_tables.sh", "fullName": "prj_yovole_ubp_ds/ubp_cal_fact_cdh_ods_tables.sh", "description": "", "children": [], "type": "FILE", "idValue": "25_0", "dirctory": false, "directory": false}, {"id": 23, "pid": 22, "name": "ubp_init_common_tables.sh", "fullName": "prj_yovole_ubp_ds/ubp_init_common_tables.sh", "description": "", "children": [], "type": "FILE", "idValue": "23_0", "dirctory": false, "directory": false}], "type": "FILE", "idValue": "22_1", "dirctory": true, "directory": true}, {"id": 29, "pid": -1, "name": "test-dir-1", "fullName": "test-dir-1", "description": "desc", "children": [{"id": 30, "pid": 29, "name": "test-dir-1-1", "fullName": "test-dir-1/test-dir-1-1", "description": "desc 1-1", "children": [{"id": 32, "pid": 30, "name": "test-dir-1-1-1", "fullName": "test-dir-1/test-dir-1-1/test-dir-1-1-1", "description": "desc 1-1-1", "children": [], "type": "FILE", "idValue": "32_1", "dirctory": true, "directory": true}], "type": "FILE", "idValue": "30_1", "dirctory": true, "directory": true}, {"id": 31, "pid": 29, "name": "test-f-1-1.sh", "fullName": "test-dir-1/test-f-1-1.sh", "description": "desc f-1-1", "children": [], "type": "FILE", "idValue": "31_0", "dirctory": false, "directory": false}], "type": "FILE", "idValue": "29_1", "dirctory": true, "directory": true}, {"id": 13, "pid": 1, "name": "aaa.sh", "fullName": "xxx.sh", "description": null, "children": [], "type": "FILE", "idValue": "13_0", "dirctory": false, "directory": false}, {"id": 21, "pid": -1, "name": "dmp_cal_fact_cmp2_cp_table.sql", "fullName": "dmp_cal_fact_cmp2_cp_table.sql", "description": "", "children": [], "type": "FILE", "idValue": "21_0", "dirctory": false, "directory": false}, {"id": 17, "pid": -1, "name": "test.sh", "fullName": "test.sh", "description": "shtestdesc", "children": [], "type": "FILE", "idValue": "17_0", "dirctory": false, "directory": false}]
""")

In [4]:
import IPython.display as disp

In [5]:
disp.JSON(resources)

<IPython.core.display.JSON object>

In [45]:
# https://stackoverflow.com/a/59109706
from itertools import islice
from colorama import Fore

space = "    "
branch = "│   "
tee = "├── "
last = "└── "


def tree_3(resources: list, level: int = -1, only_dir=False, dir_first=True, len_limit: int = 1000):
    """Given a directory Path object print a visual tree structure"""
    cnt_file = 0
    cnt_dir = 0

    def is_dir(data: dict):
        d = data.get('directory')
        return data.get('dirctory') if d is None else d

    def sort_key(data: dict):
        name = data.get('name', '{')
        if not dir_first:
            return name
        return f'0{name}' if is_dir(data) else name

    def inner(contents: list, prefix: str = "", level_=-1):
        nonlocal cnt_file, cnt_dir
        if not level_:
            return  # 0, stop iterating
        if not contents or not isinstance(contents, list):
            return
        # sort
        contents.sort(key=lambda d: sort_key(d))

        # build string
        pointers = [tee] * (len(contents) - 1) + [last]
        for pointer, path in zip(pointers, contents):
            if is_dir(path):
                yield prefix + pointer + Fore.CYAN + path['name'] + Fore.RESET
                cnt_dir += 1
                extension = branch if pointer == tee else space
                yield from inner(path.get('children'), prefix=prefix + extension, level_=level_ - 1)
            elif not only_dir:
                yield prefix + pointer + path['name']
                cnt_file += 1

    print('resources')
    iterator = inner(resources, level_=level)
    for line in islice(iterator, len_limit):
        print(line)

    if next(iterator, None):
        print(f"... len_limit, {len_limit}, reached, counted:")
    print(f"\n{cnt_dir} directories" + (f", {cnt_file} files" if cnt_file else ""))


In [46]:
tree_3(resources)

resources
├── prj_test
│   ├── aaa.sh
│   └── task1.sh
├── prj_yovole_ubp_ds
│   ├── test-cal-fact-core-asset-detail-hbase.sql
│   ├── test-cal-fact-customer-hbase.sql
│   ├── test-create-external-table.sql
│   ├── ubp_cal_fact_cdh_dmw_tables.sh
│   ├── ubp_cal_fact_cdh_ods_tables.sh
│   └── ubp_init_common_tables.sh
├── test-dir-1
│   ├── test-dir-1-1
│   │   └── test-dir-1-1-1
│   └── test-f-1-1.sh
├── aaa.sh
├── dmp_cal_fact_cmp2_cp_table.sql
└── test.sh

5 directories, 12 files


In [47]:
a = {}

In [50]:
a.pop('a', 222)

222